In [ ]:
#!pip install torch torchvision torchaudio
#!pip install einops
#!pip install datasets
#!pip install transformers
#!pip install tqdm


Biblioteki

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import h5py
from einops import rearrange
from tqdm import tqdm

import time

Wybór środowiska wykonawczego

In [10]:
#Wybór urządzenia GPU (CUDA), jeśli dostępne
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Używane urządzenie:", device)

Używane urządzenie: cpu


Podłączenie Google Drive

In [11]:
#Podłączenie Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Ustawienie parametrów eksperymentu

In [16]:
#Ścieżka do zbioru danych
file_path = '/content/drive/MyDrive/damaged_images.h5'

#Ilość próbek do pobrania
n_samples = 1000

#Wielkość segmentu danych (ilość próbek do pobrania na raz w celu oszczędzenia pamięci, np. 1.000 z 10.000)
#chunk_size = 1000
'''To rozwiązanie się nie sprawdziło, kolidowało z ładowaniem batchy'''

#Wielkość mini-batchy (ilość próbek jednoczesnie przetwarzanych przez model)
batch_size = 32

#Ilość procesów roboczych (colab ma limit 2)
n_workers = 0

#Liczba epok
n_epochs = 2

#Mieszanie kolejności próbek
shuffle = True

#Przekształcanie próbek - transformacja
transform = transforms.Compose([
    #transforms.ToPILImage(),
    #transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
'''ToPILImage prawdopodobnie nie będzie potrzebny, Resize wyłączone dla szybszego przetwarzania,
   ToTensor musi być bo chyba model nie ogarnia, że to tensory, albo ja coś źle zrozumiałem
'''

#Funkcja kosztu (przykładowo MSE)
criterion = nn.MSELoss()

#Pretrenowany model
model = models.resnet50(pretrained=True)

#Ilość warstw do Fine-tuningu (dostrajania)
fine_tune_layers = 10

Klasa ekstraktora cech

In [17]:
#Zamrożenie wszystkich warstw modelu pretrenowanego - zablokowanie możliwości trenowania wag
for param in model.parameters():
    param.requires_grad = False

#Odmrożenie n-warstw do Fine-tuningu (dostrajania)
layer_count = 0
for child in model.children():
    layer_count += 1
    if layer_count > len(list(model.children())) - fine_tune_layers:
        for param in child.parameters():
            param.requires_grad = True

class FeatureExtractor(nn.Module):
    def __init__(self, original_model):
        super(FeatureExtractor, self).__init__()
        #Odcięcie ostatniej warsatwy (klasyfikującej)
        self.features = nn.Sequential(*list(original_model.children())[:-1])

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return x

Klasa zbioru danych - nieudana

In [18]:
#Poniżej jedna z wielu prób rozwiązania problemu ilości danych i braku pamięci
#Pomysł był taki by pobierać dane co jakiś czas (np. po 1.000 próbek) aż do pewnego górnego limitu (np. 10.000)
#Dataloader miał ładować te dane do przetwarzania z okreslonym batch_size

'''
class HDF5Dataset(Dataset):
    def __init__(self, file_path, transform=None, chunk_size=1000, n_samples=None):
        self.file_path = file_path
        self.transform = transform
        self.chunk_size = chunk_size
        self.n_samples = n_samples

        # Ustalamy całkowitą liczbę próbek
        with h5py.File(self.file_path, 'r') as h5_file:
            self.total_samples = min(len(h5_file['image']), self.n_samples) if self.n_samples else len(h5_file['image'])

        self.current_chunk = None  # Zmienna przechowująca bieżący segment
        self.num_chunks = (self.total_samples + self.chunk_size - 1) // self.chunk_size  # Liczba segmentów
        self.chunk_idx = 0  # Indeks bieżącego segmentu

        # Ładujemy pierwszy segment danych
        self._load_chunk(0)

    def _load_chunk(self, chunk_idx):
        """Ładuje segment danych bazując na indeksie segmentu."""
        #start_time = time.time()

        start_idx = chunk_idx * self.chunk_size
        end_idx = min(start_idx + self.chunk_size, self.total_samples)

        with h5py.File(self.file_path, 'r') as h5_file:
            self.current_chunk = h5_file['image'][start_idx:end_idx]

        self.current_chunk_start = start_idx
        #load_time = time.time() - start_time
        #print(f"Loaded chunk {chunk_idx + 1}/{self.num_chunks} from {start_idx} to {end_idx} in {load_time:.4f} seconds")

    def __len__(self):
        return len(self.current_chunk)

    def __getitem__(self, idx):
        # Pobieranie obrazu z aktualnie załadowanego segmentu
        image = self.current_chunk[idx]

        # Ewentualne przekształcenie wymiarów
        if image.shape[0] == 3:
            image = rearrange(image, 'c h w -> h w c')

        # Ewentualne przetwarzanie za pomocą transformacji
        if self.transform:
            image = self.transform(image)

        return image

    def next_chunk(self):
        """Funkcja ładuje następny segment i zwraca, czy zostały jeszcze segmenty."""
        self.chunk_idx += 1
        if self.chunk_idx < self.num_chunks:
            self._load_chunk(self.chunk_idx)
            return True  # Oznacza, że załadowano nowy segment i można kontynuować
        else:
            self.chunk_idx = 0  # Resetujemy indeks na koniec epok
            return False  # Oznacza, że skończyliśmy wszystkie segmenty
'''

'\nclass HDF5Dataset(Dataset):\n    def __init__(self, file_path, transform=None, chunk_size=1000, n_samples=None):\n        self.file_path = file_path\n        self.transform = transform\n        self.chunk_size = chunk_size\n        self.n_samples = n_samples\n\n        # Ustalamy całkowitą liczbę próbek\n        with h5py.File(self.file_path, \'r\') as h5_file:\n            self.total_samples = min(len(h5_file[\'image\']), self.n_samples) if self.n_samples else len(h5_file[\'image\'])\n\n        self.current_chunk = None  # Zmienna przechowująca bieżący segment\n        self.num_chunks = (self.total_samples + self.chunk_size - 1) // self.chunk_size  # Liczba segmentów\n        self.chunk_idx = 0  # Indeks bieżącego segmentu\n\n        # Ładujemy pierwszy segment danych\n        self._load_chunk(0)\n\n    def _load_chunk(self, chunk_idx):\n        """Ładuje segment danych bazując na indeksie segmentu."""\n        #start_time = time.time()\n\n        start_idx = chunk_idx * self.chunk

Klasa zbioru danych

In [19]:
class HDF5Dataset(Dataset):
    def __init__(self, file_path, transform=None, n_samples=None):
        """
        Dataset do ładowania danych z pliku HDF5 i ograniczania liczby próbek.

        Args:
            file_path (str): Ścieżka do pliku HDF5.
            n_samples (int, optional): Liczba próbek, które chcemy wykorzystać.
            transform (callable, optional): Transformacje, np. augmentacja, normalizacja.
        """
        self.file_path = file_path
        self.transform = transform
        self.n_samples = n_samples

        # Otwieramy plik HDF5 raz, w konstruktorze klasy
        self.h5f = h5py.File(self.file_path, 'r')
        self.images = self.h5f['image'][:self.n_samples]  # Załadowanie grupy 'image'
        print(f"Załadowany kształt danych: {self.images.shape}")

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        # Załadowanie konkretnej próbki (obrazu) na podstawie indeksu
        image_data = self.images[idx]  # Załadowanie jednej próbki (obrazu)

        # Sprawdzamy kształt obrazu (C, H, W), jeśli konieczne, dokonujemy przekształcenia
        if image_data.shape[0] == 3:
            # Dostosowanie rozmiarów (jeśli używasz RGB, zmieniamy układ wymiarów na (h, w, c))
            image_data = rearrange(image_data, 'c h w -> h w c')  # Zmiana na (h, w, c)

        # Jeśli transformacje są zadane, stosujemy je
        if self.transform:
            image_data = self.transform(image_data)

        return image_data

    def __del__(self):
        # Zamykanie pliku HDF5, kiedy instancja obiektu jest niszczona
        self.h5f.close()

In [24]:
#Stworzenie instancji klasy FeatureExtractor i przeniesienie na środowisko wykonawcze (CPU lub GPU)
feature_extractor = FeatureExtractor(model).to(device)

#Stworzenie instancji klasy HDF5Dataset
#dataset = HDF5Dataset(file_path= file_path, transform=transform, chunk_size=chunk_size, n_samples=n_samples)
dataset = HDF5Dataset(file_path= file_path, transform=transform, n_samples=n_samples)

#Stworzenie instancji klasy DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=n_workers)

#Ustawienie modelu ekstraktora cech w tryb treningu
feature_extractor.train()

#Definicja optymalizatora dla n-douczanych warstw
params_to_train = [param for param in feature_extractor.parameters() if param.requires_grad]
optimizer = optim.Adam(params_to_train, lr=1e-4)

Załadowany kształt danych: (1000, 3, 256, 256)


In [25]:
for epoch in range(n_epochs):
    epoch_loss = 0  # Zmienna do liczenia średniego lossu na epokę
    with tqdm(dataloader, unit="batch", desc=f"Epoch {epoch+1}/{n_epochs}") as tepoch:
        for batch_idx, data in enumerate(tepoch):

            # Przenosimy dane na odpowiednie urządzenie (CPU lub GPU)
            #data = data.to(device)
            '''Włączyć gdy działamy na cuda, w innym wypadku wyłączyć to oszczędzamy jedną operację'''

            #Przekazujemy dane przez model (Feature Extractor)
            outputs = feature_extractor(data)

            # Możesz dodać obliczanie lossu, jeśli to jest wymagane
            # loss = criterion(outputs, targets)

            # Obliczanie średniego lossu na epokę (jeśli masz loss)
            # epoch_loss += loss.item()

            # Aktualizacja postępu na pasku
            tepoch.set_postfix(loss=epoch_loss / (batch_idx + 1) if batch_idx > 0 else 0)

    print(f"Epoch {epoch+1} finished.")

Epoch 1/2: 100%|██████████| 500/500 [05:31<00:00,  1.51batch/s, loss=0]


Epoch 1 finished.


Epoch 2/2:   1%|          | 5/500 [00:03<05:00,  1.65batch/s, loss=0]


KeyboardInterrupt: 

In [ ]:
#Wyniki testów (bez cuda, bo nie było dostępu gdy testowałem finalnie)
'''
Test 1:
  Parametry:
  n_samples = 1000     batch_size = 32    n_workers = 0   n_epochs = 2

  Wyniki:
  Czas wykonania:      1 epoka ~ 6 min    1 batch ~ (10 , 15) sekund

  Accuracy: (Start: -, End: -)            Loss: (Start: -, End: -)

  Uwagi:
  -
'''

'''
Test 2:
  Parametry:
  n_samples = 1000     batch_size = 32    n_workers = 2   n_epochs = 2

  Wyniki:
  Czas wykonania:      1 epoka ~ 7 min    1 batch ~ (10 , 15) sekund

  Accuracy: (Start: -, End: -)            Loss: (Start: -, End: -)

  Uwagi:
  W trakcie realizacji 2 epoki wystąpił błąd (RuntimeError: DataLoader worker (pid 48010) is killed by signal: Killed.)
  Prawdopodobnie przyczyną nie była pamięć (bo monitorowałem i była dostępna), a problem z wieloprocesowością.
  Chat sugerował by otwierać plik dopiero w __getitem__, jednak to nie rozwiązuje problemu, a wydłuża czas realizacji
'''

'''
Test 3:
  Parametry:
  n_samples = 1000     batch_size = 2    n_workers = 0   n_epochs = 2

  Wyniki:
  Czas wykonania:      1 epoka ~ 5.5 min    1 batch ~ (1.2 , 1.7) sekund

  Accuracy: (Start: -, End: -)            Loss: (Start: -, End: -)

  Uwagi:
  -
'''